In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns, numpy as np
import plotly.express as px
from pylab import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
original_df = pd.read_csv("listings.csv")
calendar = pd.read_csv("calendar.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'listings.csv'

In [ ]:
calendar.head()

In [ ]:
calendar.info()

### Data Cleaning

##### 1. Original data description

In [ ]:
print('Numer of rows and columns of the data set before cleaning: ')
original_df.shape

There are 92 features and 3,818 records in the original data set.

In [ ]:
original_df.dtypes

In [ ]:
original_df.columns

Calendar

In [ ]:
cal = calendar.copy()
cal['listing_id'].nunique()

In [ ]:
#create a new column month to extract the month from the date field
cal['mon'] = pd.DatetimeIndex(cal['date']).month
cal.groupby('mon')['listing_id'].nunique()

In [ ]:
# Strip off '$' from price since all are dollar and we need to analyze price as a number
cal.dropna(subset=['price'], inplace=True)
cal['price'] = cal['price'].str[1:]

# Replace ',' from the price and then convert it to float
cal['price'] = cal['price'].apply(lambda x: float(x.split()[0].replace(',', '')))

In [ ]:
cal.info()

##### 2. Checking duplications of the data set

In [ ]:
duplicated = original_df.duplicated()
print(duplicated)

There are no duplications in the data set.

##### 3. Checking the missing values of the data set

In [ ]:
missing = original_df.isnull().mean().round(4)
missing

In [ ]:
missing[missing > 0.2]

Columns listing above contain more than 20% of missing values.

Calendar

In [ ]:
# Drop nulls in column price since any kind of imputation may not be best here
print(len(cal))
cal.dropna(subset=['price'], inplace=True)

#validating
print(len(cal))

##### 4. Dropping columns

In [ ]:
df = original_df.drop(columns =['id','space', 'listing_url', 'scrape_id','last_scraped','experiences_offered','thumbnail_url','medium_url', 'picture_url',
                               'xl_picture_url','host_name', 'host_url','host_location','host_since','host_thumbnail_url', 'host_picture_url','host_neighbourhood',
                               'host_total_listings_count', 'host_verifications','host_has_profile_pic','street', 'neighbourhood','neighbourhood_cleansed','market',
                               'smart_location', 'country_code', 'country','bed_type','weekly_price', 'monthly_price',
                                'security_deposit','cleaning_fee', 'calendar_updated','first_review', 'last_review','requires_license','license', 'jurisdiction_names',
                               'require_guest_profile_picture','require_guest_phone_verification','calculated_host_listings_count'])

To aviod the overfitting and shift of accuracy of the model, above columns are going to be dropped off from data set. The removed columns are either unrelated features to the questions proposed or containing more than 30% of missing values.

In [ ]:
df.columns

Columns that listed above are kept for later analysis.

In [ ]:
df.info()

In [ ]:
print(df.isna().any())

In [ ]:
df_na = df.isna().sum()
df_na

In [ ]:
df.isna().sum()

Though there are still missing values including in the data set, the columns are kept for now and the missing values are going to be filled up by mean or other significant indicators. 

For the data cleaning part, we are checking the duplications and missing values from the orginal dataset. 
1. There is no duplication in the data set.
2. Columns with more than 20% missing values are listed for later processing.
2. Unrelated attributes or columns that containing more than 30% of missing values are removed.

In [ ]:
# Strip off '$' from price since all are dollar and we need to analyze price as a number
df.dropna(subset=['price'], inplace=True)
df['price'] = df['price'].str[1:]

# Replace ',' from the price and then convert it to float
df['price'] = df['price'].apply(lambda x: float(x.split()[0].replace(',', '')))

In [ ]:
print('Numer of rows and columns of the data set after cleaning: ')
df.shape

After the data cleaning, 51 features and 3,818 rows are kept for exploratory data analysis.

In [ ]:
f,ax = plt.subplots(figsize=(15,10))
df.boxplot(column=['reviews_per_month','review_scores_value','review_scores_checkin','availability_30','availability_60','availability_90'])


In [ ]:
pd.factorize(df.property_type)

In [ ]:
f,ax = plt.subplots(figsize=(20,5))
ax = sns.countplot(df.property_type,palette="muted",order = df['property_type'].value_counts().index)
plt.show()

<b>Most of the property types are House and Apartment around 1700 each.</b>

In [ ]:
f,ax = plt.subplots(figsize=(15,10))
df.isna().sum()[df.isna().sum()>0].plot(kind='bar')

Square Feet column has the most null values

In [ ]:
f,ax = plt.subplots(figsize=(25,10))
ax = sns.scatterplot(y=df.latitude,x=df.longitude,hue=df.neighbourhood_group_cleansed,palette="coolwarm")
plt.show()

Queen Anne, Lake City and Capitol Hill has highest number of Airbnbs.

In [ ]:
f,ax = plt.subplots(figsize=(15,5))
ax = sns.distplot(df.availability_30)
plt.show()

In [ ]:
sns.set(rc={"figure.figsize": (16, 8)});
np.random.seed(0)
x = np.random.randn(100)
subplot(2,2,1)
ax = sns.distplot(df.availability_30)

subplot(2,2,2)
ax = sns.distplot(df.availability_60)

subplot(2,2,3)
ax = sns.distplot(df.availability_90)

subplot(2,2,4)
ax = sns.distplot(df.availability_365)

plt.show()


So of the properties are available throughout the year.

# We want to understand whether the reviews support the ratings or not. Is there any discrepancy between ratings and reviews? 

In [ ]:
df.corr()

In [ ]:
# Find the relationship between review score accuracy and review score rating
# Plot the HeatMap to see which variables have the strong correlationships 
plt.figure(figsize = (20,15))
heat_map = sns.heatmap(df.corr(), annot = True, linewidth = 1.5, linecolor = "black", fmt = ".2f", center = 0, vmin = -1, vmax = 1, cmap = "RdBu")
plt.title("HeatMap using Seaborn Method", size = 25)
plt.show()

In [ ]:
# Check all variables to see which variable has the strongest correlationship with "review_scores_accuracy"
df.corr()["review_scores_accuracy"].sort_values(ascending = False)[df.corr()["review_scores_accuracy"].sort_values() > 0].head(10)

The result tells us that "review_scores_rating" has the strongest correlationship to "review_scores_accuracy". 
The next leading factors are "review_scores_value", and "review_scores_cleanliness".

# Which variables can affect the price? 

In [ ]:
fig = px.box(df, y="price", color = "accommodates")
fig.show()

In [ ]:
df.info()

In [ ]:
# Check all variables to see which variable has the strongest correlationship with "price"
df.corr()['price'].sort_values(ascending=False)[df.corr()['price'].sort_values()>0].head(11)

We found out that accomodates, bedrooms, and beds are the top three variables that have the most correlation with price.

In [ ]:
df.corr()['price'].sort_values()[df.corr()['price'].sort_values()<0].head(10)

We found out that reviews_per_month, Number_of_reviews and longitude are the top three variables that have the negative correlation with price

# Busiest time to visit Seattle

In [ ]:
# Unique count of the houses listed in a month
mon_avail = cal.groupby('mon')['listing_id'].nunique()
mon_avail

# # Plot the number of unique houses for every month
plt.figure(figsize=[12,8]);
ax = sns.barplot(mon_avail.index,mon_avail).set(title='Busiest times to visit Seattle')
plt.xlabel('Month');
plt.ylabel('Number of houses available');
plt.show()


 We observe that:
- From a total of 3818 house those were listed on Airbnb, 3600 houses are available in Jan for at least a day. 
- We also notice that July and August has the minimum number of houses ~2500 available for at least a day.

Hence, we can conclude that July and August are the busiest times to visit Seattle. As observed from the trend,Summer generally attracts more people hence is busiest period of the year to visit Seattle.

In [ ]:
# Calculate the spike in the prices per month
mon_avg_price = cal.groupby('mon')['price'].mean()
mon_avg_price
# # Plot the average monthly price for all months
plt.figure(figsize=[12,6]);
plt.plot(mon_avg_price.index, mon_avg_price);
plt.xlabel('Month');
plt.ylabel('Avg. Price');
plt.title('Montly spike in the prices observed');

We observe that:
- Trends show that average price increased during Summer(June,July,Aug) with a peak observed during July with average price of around $152. 
- After the peak in July, the average price continues to decrease steadily until November after that it starts to increase again.

# What are the characteristics of the high rating properties?

In [ ]:
df["review_scores_rating"].describe()

In [ ]:
ax = sns.distplot(df.review_scores_rating)

According to the descriptive statistics and the frequency distribution above, the average review scores are about 94.54 points and more than 17.5% properties (with total 3171 properties) received 100 points.
Properties that received 100 points are set as high rating properties.

To find out the high rating properties' charateristics, we are going to compare the statistics between properties that received full points to those properties that did not recieve full marks.

##### Overview

In [ ]:
high_rating = df[(df.review_scores_rating == 100)] 
other_rating = df[(df.review_scores_rating != 100)]

In [ ]:
high_rating.describe()

In [ ]:
other_rating.describe()

##### Property type

In [ ]:
# property_type
fig, axes = plt.subplots(nrows=2, ncols=1)
sns.countplot(high_rating.property_type,palette="muted",order = high_rating['property_type'].value_counts().index, ax=axes[0])
sns.countplot(other_rating.property_type,palette="muted",order = other_rating['property_type'].value_counts().index, ax=axes[1])
plt.show()

##### Conditions

In [ ]:
# accommodates
fig, axes = plt.subplots(nrows=1, ncols=2)
sns.countplot(high_rating.accommodates,palette="muted",order = high_rating['accommodates'].value_counts().index, ax=axes[0])
sns.countplot(other_rating.accommodates,palette="muted",order = other_rating['accommodates'].value_counts().index, ax=axes[1])
plt.show()

In [ ]:
# bathrooms
fig, axes = plt.subplots(nrows=1, ncols=2)
sns.countplot(high_rating.bathrooms,palette="muted",order = high_rating['bathrooms'].value_counts().index, ax=axes[0])
sns.countplot(other_rating.bathrooms,palette="muted",order = other_rating['bathrooms'].value_counts().index, ax=axes[1])
plt.show()

In [ ]:
# bedrooms
fig, axes = plt.subplots(nrows=1, ncols=2)
sns.countplot(high_rating.bedrooms,palette="muted",order = high_rating['bedrooms'].value_counts().index, ax=axes[0])
sns.countplot(other_rating.bedrooms,palette="muted",order = other_rating['bedrooms'].value_counts().index, ax=axes[1])
plt.show()

In [ ]:
# beds
fig, axes = plt.subplots(nrows=1, ncols=2)
sns.countplot(high_rating.beds,palette="muted",order = high_rating['beds'].value_counts().index, ax=axes[0])
sns.countplot(other_rating.beds,palette="muted",order = other_rating['beds'].value_counts().index, ax=axes[1])
plt.show()

In [ ]:
# Square_feet
high_rating['square_feet'].describe()

In [ ]:
other_rating['square_feet'].describe()

##### Booking and cancel aspects

In [ ]:
# instant_bookable
fig, axes = plt.subplots(nrows=1, ncols=2)
sns.countplot(high_rating.instant_bookable,palette="muted",order = high_rating['instant_bookable'].value_counts().index, ax=axes[0])
sns.countplot(other_rating.instant_bookable,palette="muted",order = other_rating['instant_bookable'].value_counts().index, ax=axes[1])
plt.show()

In [ ]:
# cancellation_policy
fig, axes = plt.subplots(nrows=1, ncols=2)
sns.countplot(high_rating.cancellation_policy,palette="muted",order = high_rating['cancellation_policy'].value_counts().index, ax=axes[0])
sns.countplot(other_rating.cancellation_policy,palette="muted",order = other_rating['cancellation_policy'].value_counts().index, ax=axes[1])
plt.show()

##### Avaibility

In [ ]:
# availability_30 
fig, axes = plt.subplots(nrows=1, ncols=2)
sns.countplot(high_rating.availability_30,palette="muted",order = high_rating['availability_30'].value_counts().index, ax=axes[0])
sns.countplot(other_rating.availability_30,palette="muted",order = other_rating['availability_30'].value_counts().index, ax=axes[1])
plt.show()

##### Price

In [ ]:
# price
high_rating['price'].str.slice(start=1).str.replace(',', '').astype(float).describe()

In [ ]:
other_rating['price'].str.slice(start=1).str.replace(',', '').astype(float).describe()

##### Location

In [ ]:
# location 
fig, axes = plt.subplots(nrows=2, ncols=1)
sns.scatterplot(y=high_rating.latitude,x=high_rating.longitude,hue=high_rating.neighbourhood_group_cleansed,palette="coolwarm",ax=axes[0])
sns.scatterplot(y=other_rating.latitude,x=other_rating.longitude,hue=other_rating.neighbourhood_group_cleansed,palette="coolwarm",ax=axes[1])
plt.show()

### Review Sentiment Analysis

In [ ]:
#import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
listings = pd.read_csv("/Users/snehaltikone/Spring_2022/DataMining/Project/archive/listings.csv")

In [ ]:
## Creating a dataset with neiborhood reviews and area
neighbourhood = listings[['neighborhood_overview','neighbourhood','review_scores_rating']]

In [ ]:
#Checing for null values
neighbourhood.isna().sum()

In [ ]:
#Deleting rows with null values
neighbourhood=neighbourhood.dropna()

In [ ]:
#After dropping rows with null values
neighbourhood.isnull()

In [ ]:
##Here begins the fun
#Stop words is pre-defined python list of common english words which we never consider for any analysis. 
#We can update the stopword list

stopwords = set(STOPWORDS)
stopwords.update(["br", "href","Ballard","Capitol Hill","Queen Anne","Seattle","Capitol Hill","home","Fremont"])
textt = " ".join(review for review in neighbourhood.neighborhood_overview) 
wordcloud = WordCloud(stopwords=stopwords).generate(textt)


fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud11.png')
plt.show()

### Categorizing the reviews 

In [ ]:
# assign reviews with score > 85 as positive sentiment
# score < 85 negative sentiment
# remove score = 85
neighbourhood = neighbourhood[neighbourhood['review_scores_rating'] != 85]
neighbourhood['sentiment'] = neighbourhood['review_scores_rating'].apply(lambda rating : +1 if rating > 85 else -1)

In [ ]:
# split df - positive and negative sentiment:
positive = neighbourhood[neighbourhood['sentiment'] == 1]
negative = neighbourhood[neighbourhood['sentiment'] == -1]

### Positive Reviews Bundle

In [ ]:
stopwords = set(STOPWORDS)
stopwords.update(["br", "href","sad","creepy","area","city","walk","restaurant",'restaurants',"neighborhood","Seattle",'downtown','minute','Park','Capitol', 'Hill','house','home','Fremont',' Capitol Hill']) 
## good and great removed because they were included in negative sentiment
pos = " ".join(review for review in positive.neighborhood_overview)
wordcloud2 = WordCloud(stopwords=stopwords,background_color='skyblue').generate(pos)
plt.imshow(wordcloud2, interpolation='bilinear')
plt.axis("off")
plt.show()

### Negative Reviews Bundle

In [ ]:
neg = " ".join(review for review in negative.neighborhood_overview)
stopwords.update(["br", "href","sad","creepy","area","city","walk","restaurant",'restaurants',"neighborhood","Seattle",'downtown','minute','Park','Capitol Hill','house','home','Fremont','great','located','bar',
'shop','Capitol  Hill','food','Pike Place','shops','Capitol','Hill','best','will','close','one','many']) 
wordcloud3 = WordCloud(stopwords=stopwords,background_color='white').generate(neg)
plt.imshow(wordcloud3, interpolation='bilinear')
plt.axis("off")
plt.savefig('wordcloud33.png')
plt.show()

## Positive Reviews Vs Negative reviews

In [ ]:
neighbourhood['sentimentt'] = neighbourhood['sentiment'].replace({-1 : 'negative'})
neighbourhood['sentimentt'] = neighbourhood['sentimentt'].replace({1 : 'positive'})
fig = px.histogram(neighbourhood, x="sentimentt",labels=dict(sentimentt="Reviews"))
fig.update_traces(marker_color="pink",marker_line_color='rgb(8,48,107)', marker_line_width=1.5)
fig.update_layout(title_text='Airbnb Reviews Sentiments',title_x=0.5)
fig.show()

In [ ]:
len(neighbourhood) ## since neighborhood is a tuple we cannot use shape hence used len()

In [ ]:
2047/2189

In [ ]:
142/2189

## End